<a href="https://colab.research.google.com/github/vikrantpotnis123/DS/blob/master/cnn1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision 
import torchvision.transforms as transforms
torch.set_printoptions(linewidth=120)

In [ ]:
# init params
train_set = torchvision.datasets.FashionMNIST(
    root = './data/FashionMNIST',
    train = True,
    download=True,
    transform = transforms.Compose([
      transforms.ToTensor()                 
    ]   
    )
)
train_loader = torch.utils.data.DataLoader(
    train_set, batch_size = 64, shuffle=True
)


test_set = torchvision.datasets.FashionMNIST(
    root = './data/FashionMNIST',
    train = False,
    download = False,
    transform = transforms.Compose([
      transforms.ToTensor()                 
    ]   
    )
)
test_loader = torch.utils.data.DataLoader(
    test_set, batch_size = 64, shuffle=True
)

#device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#hyper paramters
num_classes = 10
num_epochs  = 15
batch_size = 4
learning_rate = 0.001

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
torch.set_printoptions(linewidth=120)

In [ ]:
len(train_set)

In [ ]:
train_set.targets

In [ ]:
train_set.targets.bincount()


In [ ]:
sample = next(iter(train_set))
print(len(sample))
type(sample)

In [ ]:
image, label = sample
print(image.shape)
print(label)
#label.shape

In [ ]:
plt.imshow(image.squeeze(), cmap='gray')
print('label: ', label)

In [ ]:
batch = next(iter(train_loader))
len(batch)

In [ ]:
type(batch)

In [ ]:
images, labels = batch

In [ ]:
labels.shape

In [ ]:
images.shape

In [ ]:
grid = torchvision.utils.make_grid(images, nrow=10)
plt.figure(figsize=(15,15))
plt.imshow(np.transpose(grid, (1, 2 ,  0)))
print('labels: ', labels)

In [ ]:
class ConvNet(nn.Module):
  
    def __init__(self):
      super().__init__()
      self.conv1 = nn.Conv2d(1, 20, 5, 1)
      self.conv2 = nn.Conv2d(20, 50, 5, 1)
      self.fc1 = nn.Linear(4*4*50, 500)
      self.dropout1 = nn.Dropout(0.5)
      self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
      x = F.relu(self.conv1(x))
      x = F.max_pool2d(x, 2, 2)
      x = F.relu(self.conv2(x))
      x = F.max_pool2d(x, 2, 2)
      x = x.view(-1, 4*4*50)
      x = F.relu(self.fc1(x))
      x = self.dropout1(x)
      x = self.fc2(x)
      return x
    

In [ ]:
model = ConvNet().to(device)
model

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [ ]:
n_total_steps = len(train_loader) # 600
for e in range(num_epochs):
  n_loss = 0.0
  n_correct = 0.0

  for i, (images, labels) in enumerate(train_loader): 
    images = images.to(device)
    labels = labels.to(device)
    output= model(images)
    loss = criterion(output, labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # print every 100th iteration
    if ((i+1)%100 == 0):
      print(f'epoch {e+1}/{num_epochs}, step {i*1}/{n_total_steps}, loss = {loss.item():.4f}')
  else:
    with torch.no_grad():
      n_correct = 0.0
      n_samples = 0.0

      for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        output = model(images)
        _, preds = torch.max(output, 1)
        #print(labels.shape[0])
        # batch_size
        n_samples += labels.shape[0]

        # correct preds within a batch_size of samples
        n_correct += (preds == labels).sum().item() 

        # overall accuracy
        acc = 100.0 * n_correct/n_samples
        print(f'epoch {e+1}/{num_epochs}, accuracy = {acc}')
    